In [117]:
import pandas as pd
import numpy as np
import keras as K
from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.regularizers import l2
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback, TensorBoard

In [148]:
PATH = '/Users/krishnakalyan3/Educational/FastAI/code/data/ml-latest-small/ratings.csv'

In [149]:
ratings = pd.read_csv(PATH)

In [150]:
ratings.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [151]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [152]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [153]:
ratings.tail()

,userId,movieId,rating,timestamp
99999,670,7005,2.5,1065579370
100000,670,4771,4.0,1065149201
100001,670,1329,4.0,1070940363
100002,670,1331,2.5,1070979663
100003,670,2946,3.5,1074784724


In [135]:
np.random.seed = 42
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [136]:
def rmse(y_true, y_pred):
    score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
    return score

In [137]:
from keras.layers.merge import concatenate

In [138]:
def model():
    u_ip = Input(shape=(1,), dtype='int64', name='user')
    m_ip = Input(shape=(1,), dtype='int64', name='movies')
    
    u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-3))(u_ip)
    m = Embedding(n_articles, n_factors, input_length=1, embeddings_regularizer=l2(1e-3))(m_ip)
    
    x = concatenate([u, m])
    x = Flatten()(x)

    x = Dropout(0.5)(x)
    x = Dense(1000, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1)(x)
    
    model = Model([u_ip, m_ip], x)
    model.compile(Adam(5e-4), loss='mse', metrics=[rmse])
    return model

In [139]:
model = model()

In [141]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=2, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80031 samples, validate on 19973 samples
Epoch 1/2
  448/80031 [..............................] - ETA: 32s - loss: 0.8458 - rmse: 0.8776

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


80031/80031 [==============================] - 27s - loss: 0.8889 - rmse: 0.8969 - val_loss: 0.8723 - val_rmse: 0.8765
Epoch 2/2
80031/80031 [==============================] - 27s - loss: 0.8738 - rmse: 0.8859 - val_loss: 0.8625 - val_rmse: 0.8688
